In [2]:
import pdfminer
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
from io import StringIO
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

def extract_pages_to_xml(input_pdf, output_xml, start_page, end_page, title):
    # PDF에서 텍스트 추출
    output = StringIO()
    with open(input_pdf, 'rb') as pdf_file:
        extract_text_to_fp(pdf_file, output, laparams=LAParams(), 
                           output_type='xml', codec=None, 
                           page_numbers=range(start_page-1, end_page))
    
    # XML 파싱
    root = ET.fromstring(output.getvalue())
    
    # 새 XML 구조 생성
    new_root = ET.Element("document")
    ET.SubElement(new_root, "title").text = title
    ET.SubElement(new_root, "start_page").text = str(start_page)
    ET.SubElement(new_root, "end_page").text = str(end_page)
    content = ET.SubElement(new_root, "content")
    
    # 페이지별로 내용 추가
    for page in root.findall(".//page"):
        page_elem = ET.SubElement(content, "page")
        page_elem.set("number", page.get("id"))
        for text in page.findall(".//text"):
            ET.SubElement(page_elem, "text").text = text.text
    
    # XML 저장
    xml_str = minidom.parseString(ET.tostring(new_root)).toprettyxml(indent="  ")
    with open(output_xml, "w", encoding="utf-8") as f:
        f.write(xml_str)

# 사용 예시
input_pdf = './20240915190844072.pdf'

extract_pages_to_xml(input_pdf, '교내전화번호안내.xml', 1, 1, "교내전화번호안내")

In [4]:
import os
import xml.etree.ElementTree as ET

def xml_to_markdown(xml_file_path, output_dir):
    # XML 파일 파싱
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Markdown 문자열 초기화
    markdown = ""

    # 제목 추가
    title = root.find('title').text
    markdown += f"# {title}\n\n"

    # 페이지 범위 추가
    start_page = root.find('start_page').text
    end_page = root.find('end_page').text
    markdown += f"페이지 범위: {start_page} - {end_page}\n\n"

    # 내용 추가
    content = root.find('content')
    for page in content.findall('page'):
        page_number = page.get('number')
        markdown += f"## 페이지 {page_number}\n\n"
        for text in page.findall('text'):
            if text.text:
                markdown += f"{text.text}\n\n"

    # Markdown 파일 저장
    base_name = os.path.splitext(os.path.basename(xml_file_path))[0]
    output_file_path = os.path.join(output_dir, f"{base_name}.md")
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(markdown)

    print(f"Created: {output_file_path}")

def convert_all_xmls(input_dir, output_dir):
    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 모든 XML 파일에 대해 변환 수행
    for filename in os.listdir(input_dir):
        if filename.endswith(".xml"):
            xml_file_path = os.path.join(input_dir, filename)
            xml_to_markdown(xml_file_path, output_dir)

# 스크립트 실행
input_dir = r"./xmls"
output_dir = r"./markdowns"
convert_all_xmls(input_dir, output_dir)

Created: ./markdowns\K-MOOC 교내 온라인강좌 수강 안내.md
Created: ./markdowns\공학교육인증제.md
Created: ./markdowns\교양 필수 교과목 수강 안내.md
Created: ./markdowns\다전공 안내.md
Created: ./markdowns\다학년 다학기 프로젝트 (KW-VIP) 안내.md
Created: ./markdowns\매치업 집중이수제 교내 강좌 수강 안내.md
Created: ./markdowns\서비스러닝 교과목 안내.md
Created: ./markdowns\수강신청.md
Created: ./markdowns\연계전공 안내.md
Created: ./markdowns\졸업이수학점 안내 (19년 이후 신입학자).md
Created: ./markdowns\졸업이수학점 안내 (공학인증학과, 19년 이후 신입학자).md
Created: ./markdowns\졸업이수학점 안내 (다전공, 19년 이후 신입학자).md
Created: ./markdowns\참빛설계학기 안내.md
Created: ./markdowns\특별교육과정 (CDP, R.O.T.C., 글로벌인재트랙인증제).md
Created: ./markdowns\편입생이 이수해야 할 교과목 및 학점.md
Created: ./markdowns\학-석사 연계과정.md
Created: ./markdowns\현장실습학기제 안내.md
